In [ ]:
import pandas as pd
import glob
import os
from IPython.display import HTML, display
from cellphonedb.utils import db_releases_utils
from cellphonedb.utils import db_utils
import anndata
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method
import ktplotspy as kpy

In [ ]:
#process metadat for combining endo cell type
metadata_d = pd.read_csv('rawTXT/decidua_downsample10k_metadata_cutted.txt', sep='\t')
#metadata_d['cell_type2'] = list(metadata_d['cell_type'])
metadata_d['cell_type'] = metadata_d['cell_type'].replace('Endo (m)','Endo')
metadata_d['cell_type'] = metadata_d['cell_type'].replace('Endo L','Endo')
metadata_d['cell_type'] = metadata_d['cell_type'].replace('Endo (f)','Endo')
metadata_d['cell_type'].value_counts()

metadata_d.to_csv('rawTXT/decidua_downsample10k_metadata_cutted2.txt', sep='\t',index=False)

In [ ]:
cpdb_file_path = 'db/v4.1.0/cellphonedb.zip'
meta_file_path = 'rawTXT/decidua_downsample10k_metadata_cutted2.txt'
counts_file_path = 'rawTXT/decidua_downsample10k_filteredGenes_lognorm.h5ad'
out_path = 'imputed_res/rawLogNorm/decidua'

In [ ]:
metadata = pd.read_csv(meta_file_path, sep='\t')
metadata.head(3)

In [ ]:
adata = anndata.read_h5ad(counts_file_path)
#adata.obs_names.name = 'barcode_sample'
adata.shape

In [ ]:
print(adata.obs_names[1:5])
print(adata.var_names[1:5])

In [ ]:
adata[0:5,0:5].to_df()

In [ ]:
#list(adata.obs.index).sort() == list(metadata['barcode_sample']).sort()
list(adata.obs.index)==list(metadata['barcode_sample'])

In [ ]:
deconvoluted, means, pvalues, significant_means = cpdb_statistical_analysis_method.call(
    cpdb_file_path = cpdb_file_path,                 # mandatory: CellPhoneDB database zip file.
    meta_file_path = meta_file_path,                 # mandatory: tsv file defining barcodes to cell label.
    counts_file_path = counts_file_path,             # mandatory: normalized count matrix.
    counts_data = 'hgnc_symbol',                     # defines the gene annotation in counts matrix.
    #microenvs_file_path = microenvs_file_path,       # optional (default: None): defines cells per microenvironment.
    iterations = 1000,                               # denotes the number of shufflings performed in the analysis.
    threshold = 0.1,                                 # defines the min % of cells expressing a gene for this to be employed in the analysis.
    threads = 4,                                     # number of threads to use in the analysis.
    debug_seed = 42,                                 # debug randome seed. To disable >=0.
    result_precision = 3,                            # Sets the rounding for the mean values in significan_means.
    pvalue = 0.05,                                   # P-value threshold to employ for significance.
    subsampling = False,                             # To enable subsampling the data (geometri sketching).
    subsampling_log = False,                         # (mandatory) enable subsampling log1p for non log-transformed data inputs.
    subsampling_num_pc = 100,                        # Number of componets to subsample via geometric skectching (dafault: 100).
    subsampling_num_cells = 10000,                    # Number of cells to subsample (integer) (default: 1/3 of the dataset).
    separator = '|',                                 # Sets the string to employ to separate cells in the results dataframes "cellA|CellB".
    debug = False,                                   # Saves all intermediate tables employed during the analysis in pkl format.
    output_path = out_path,                          # Path to save results.
    output_suffix = None                             # Replaces the timestamp in the output files by a user defined string in the  (default: None).
    )

In [ ]:
#afMF process decidua
meta_file_path = 'test_CellPhoneDB/rawTXT/decidua_downsample10k_metadata_cutted.txt'
counts_file_path = 'afMF/imputed_data/decidua_downsample10k_filteredGenes_sigma3_0_convergence_True.txt'
metadata = pd.read_csv(meta_file_path, sep='\t')
print(metadata.head(3))
adata = anndata.read_csv(counts_file_path,delimiter="\t")
print(adata.shape)
adata = adata.transpose()
adata.obs_names.name = 'barcode_sample'
print(adata.obs_names[1:5])
print(adata.var_names[1:5])
print(adata[0:5,0:5].to_df())
adata.write_h5ad('afMF/imputed_data/decidua_downsample10k_filteredGenes.h5ad')

In [ ]:
#run cpdb
cpdb_file_path = 'test_CellPhoneDB/db/v4.1.0/cellphonedb.zip'
meta_file_path = 'test_CellPhoneDB/rawTXT/decidua_downsample10k_metadata_cutted2.txt'
out_path = 'test_CellPhoneDB/imputed_res/afMF/decidua'
counts_file_path = 'afMF/imputed_data/decidua_downsample10k_filteredGenes.h5ad'

In [ ]:
deconvoluted, means, pvalues, significant_means = cpdb_statistical_analysis_method.call(
    cpdb_file_path = cpdb_file_path,                 # mandatory: CellPhoneDB database zip file.
    meta_file_path = meta_file_path,                 # mandatory: tsv file defining barcodes to cell label.
    counts_file_path = counts_file_path,             # mandatory: normalized count matrix.
    counts_data = 'hgnc_symbol',                     # defines the gene annotation in counts matrix.
    #microenvs_file_path = microenvs_file_path,       # optional (default: None): defines cells per microenvironment.
    iterations = 1000,                               # denotes the number of shufflings performed in the analysis.
    threshold = 0.1,                                 # defines the min % of cells expressing a gene for this to be employed in the analysis.
    threads = 4,                                     # number of threads to use in the analysis.
    debug_seed = 42,                                 # debug randome seed. To disable >=0.
    result_precision = 3,                            # Sets the rounding for the mean values in significan_means.
    pvalue = 0.05,                                   # P-value threshold to employ for significance.
    subsampling = False,                             # To enable subsampling the data (geometri sketching).
    subsampling_log = False,                         # (mandatory) enable subsampling log1p for non log-transformed data inputs.
    subsampling_num_pc = 100,                        # Number of componets to subsample via geometric skectching (dafault: 100).
    subsampling_num_cells = 10000,                    # Number of cells to subsample (integer) (default: 1/3 of the dataset).
    separator = '|',                                 # Sets the string to employ to separate cells in the results dataframes "cellA|CellB".
    debug = False,                                   # Saves all intermediate tables employed during the analysis in pkl format.
    output_path = out_path,                          # Path to save results.
    output_suffix = None                             # Replaces the timestamp in the output files by a user defined string in the  (default: None).
    )